In [1]:
from Spreadsheet.Spreadsheet import Spreadsheet

spreadsheet = Spreadsheet("pito.s2v")

spreadsheet.set("A1", "Hello") 

spreadsheet.set("A2", "2")

print(spreadsheet.get("A1"))

print(spreadsheet.get("A2"))


Hello
text
2
number
Hello
2.0


In [2]:
import re

class Tokenizer:
    def __init__(self, formula):
        self.formula = formula

    def tokenize(self):
        # Regular expression patterns for different token types
        pattern = {
            'operator': r'\+|-|\*|/',
            'coordinate': r'[A-Z]+\d+',
            'number': r'\d+\.?\d*',
            'opening_round_bracket': r'\(',
            'closing_round_bracket': r'\)',
            'colon': r':',
            'semi_colon': r';',
            'function': r'SUMA|PROMEDIO|MAX|MIN',  # Additional functions can be added as necessary
            'range': r'[A-Z]+\d+:[A-Z]+\d+'  # For example, A1:B3
        }

        # Combine the patterns into a single regular expression
        combined_pattern = '|'.join(f'({p})' for p in pattern.values())
        
        # Find all matches of the pattern in the formula
        tokens = re.findall(combined_pattern, self.formula)
        
        # Flatten the list of tuples and filter out empty strings
        tokens = [token for token in sum(tokens, ()) if token != '']

        return tokens

# Example usage
formula = "=1 + A1*((SUMA(A2:B5;PROMEDIO(B6:D8);C1;27)/4)+(D6-D8))"
tokenizer = Tokenizer(formula)
print(tokenizer.tokenize())


['1', '+', 'A1', '*', '(', '(', 'SUMA', '(', 'A2', ':', 'B5', ';', 'PROMEDIO', '(', 'B6', ':', 'D8', ')', ';', 'C1', ';', '27', ')', '/', '4', ')', '+', '(', 'D6', '-', 'D8', ')', ')']


In [3]:
import re

class Tokenizer:
    def __init__(self, formula):
        self.formula = formula

    def tokenize(self):
        # Regular expression patterns for different token types
        pattern = {
            'operator': r'\+|-|\*|/',
            'coordinate': r'[A-Z]+\d+',
            'number': r'\d+\.?\d*',
            'opening_round_bracket': r'\(',
            'closing_round_bracket': r'\)',
            'colon': r':',
            'semi_colon': r';',
            'function': r'SUMA|PROMEDIO|MAX|MIN',  # Additional functions can be added as necessary
            'range': r'[A-Z]+\d+:[A-Z]+\d+'  # For example, A1:B3
        }

        # Combine the patterns into a single regular expression
        combined_pattern = '|'.join(f'({p})' for p in pattern.values())
        
        # Find all matches of the pattern in the formula
        tokens = re.findall(combined_pattern, self.formula)
        
        # Flatten the list of tuples and filter out empty strings
        tokens = [token for token in sum(tokens, ()) if token != '']

        return tokens
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_index = 0
        self.function_stack = []  # Stack to track function context

    def next_token(self):
        if self.current_index < len(self.tokens):
            token = self.tokens[self.current_index]
            self.current_index += 1
            return token
        else:
            return None

    def parse(self):
        if not self.tokens:
            raise ValueError("No tokens to parse")

        last_token_type = None
        parenthesis_count = 0
        expecting_range = False  # Flag to track if a colon for a range is expected

        while self.current_index < len(self.tokens):
            token = self.next_token()
            #print(token)
            if self.is_function(token):
                if last_token_type in ['operand', 'closing_round_bracket']:
                    raise SyntaxError("Function name cannot follow an operand or closing bracket directly")
                inside_function = True
                self.function_stack.append(token)
                last_token_type = 'function'
            elif self.is_coordinate(token):
                # Handle cell coordinates
                if last_token_type == 'operand' and not expecting_range:
                    raise SyntaxError("Two consecutive operands")
                last_token_type = 'operand'
                expecting_range = True  # After a coordinate, a colon for range is valid
            elif token == ';':
                # Handle semicolon as function argument separator
                if not self.function_stack or last_token_type in ['operator', 'opening_round_bracket', ';']:
                    raise SyntaxError("Invalid use of ';' as argument separator")
                last_token_type = ';'
            elif self.is_number(token):
                # Handle numbers (including decimals)
                if last_token_type in ['operand', 'closing_round_bracket']:
                    raise SyntaxError("Invalid syntax: number follows an operand or closing bracket without an operator")
                last_token_type = 'operand'
                expecting_range = False
            elif token == ':':
                # Handle colon for cell ranges
                if not expecting_range or last_token_type != 'operand':
                    raise SyntaxError("Invalid use of ':' for cell range")
                last_token_type = 'range'
                expecting_range = False  # Reset the flag as colon is consumed
            elif token in '+-*/':
                if last_token_type in [None, 'operator', '(']:
                    raise SyntaxError("Operator in invalid position")
                last_token_type = 'operator'
            elif token == '(':
                parenthesis_count += 1
                if last_token_type in ['operand', 'closing_round_bracket']:
                    raise SyntaxError("Operand or closing bracket followed directly by '('")
                last_token_type = '('
            elif token == ')':
                parenthesis_count -= 1
                if last_token_type in ['operator', '(']:
                    raise SyntaxError("')' cannot follow an operator or '('")
                last_token_type = ')'
                if self.function_stack:
                    self.function_stack.pop()
            else:
                raise SyntaxError(f"Unknown token: {token}")

            if parenthesis_count < 0:
                raise SyntaxError("Unbalanced parentheses")

        if parenthesis_count != 0:
            raise SyntaxError("Unbalanced parentheses")

        return self.tokens

    def is_coordinate(self, token):
        # A cell coordinate is defined as one or more uppercase letters followed by one or more digits
        coordinate_pattern = r'^[A-Z]+\d+$'
        #print(re.match(coordinate_pattern, token) is not None)
        return re.match(coordinate_pattern, token) is not None

    def is_function(self, token):
        # List of valid function names
        valid_functions = ["SUMA", "PROMEDIO", "MAX", "MIN"]
        return token in valid_functions
    
    def is_number(self, token):
        number_pattern = r'^\d+\.?\d*$'
        return re.match(number_pattern, token) is not None

# Example usage
    
# Example usage
formula = "=1.05 + A1*((SUMA(A2:B5;PROMEDIO(B6:D8);C1;27)/4)+(D6-D8))"
tokenizer = Tokenizer(formula)
tokens = tokenizer.tokenize()
print(tokens)
parser = Parser(tokens)
try:
    parsed_tokens = parser.parse()
    print("Syntax is correct:", parsed_tokens)
except SyntaxError as e:
    print("Syntax error:", e)

['1.05', '+', 'A1', '*', '(', '(', 'SUMA', '(', 'A2', ':', 'B5', ';', 'PROMEDIO', '(', 'B6', ':', 'D8', ')', ';', 'C1', ';', '27', ')', '/', '4', ')', '+', '(', 'D6', '-', 'D8', ')', ')']
Syntax is correct: ['1.05', '+', 'A1', '*', '(', '(', 'SUMA', '(', 'A2', ':', 'B5', ';', 'PROMEDIO', '(', 'B6', ':', 'D8', ')', ';', 'C1', ';', '27', ')', '/', '4', ')', '+', '(', 'D6', '-', 'D8', ')', ')']


In [4]:
class GeneratePostfix:
    def __init__(self, tokens):
        self.tokens = tokens
        self.output_queue = []
        self.operator_stack = []
        # Update the precedence for ':' and ';'
        self.operators = {'+': 1, '-': 1, '*': 2, '/': 2, ':': 10, ';': 1}
        self.functions = {'SUMA', 'PROMEDIO', 'MAX', 'MIN'}

    def precedence(self, op):
        return self.operators.get(op, 0)

    def generate_postfix(self):
        for token in self.tokens:
            if self.is_number(token) or self.is_coordinate(token):
                self.output_queue.append(token)
            elif token in self.operators:
                while self.operator_stack and self.operator_stack[-1] != '(' and \
                      self.precedence(self.operator_stack[-1]) >= self.precedence(token):
                    self.output_queue.append(self.operator_stack.pop())
                self.operator_stack.append(token)
            elif token in self.functions:
                self.operator_stack.append(token)
            elif token == '(':
                self.operator_stack.append(token)
            elif token == ')':
                while self.operator_stack and self.operator_stack[-1] != '(':
                    self.output_queue.append(self.operator_stack.pop())
                self.operator_stack.pop()  # Pop '(' from stack
                if self.operator_stack and self.operator_stack[-1] in self.functions:
                    self.output_queue.append(self.operator_stack.pop())

        while self.operator_stack:
            self.output_queue.append(self.operator_stack.pop())

        return self.output_queue

    def is_coordinate(self, token):
        # A cell coordinate is defined as one or more uppercase letters followed by one or more digits
        coordinate_pattern = r'^[A-Z]+\d+$'
        #print(re.match(coordinate_pattern, token) is not None)
        return re.match(coordinate_pattern, token) is not None
    
    def is_number(self, token):
        number_pattern = r'^\d+\.?\d*$'
        return re.match(number_pattern, token) is not None

# Example usage
formula = "A4-B2/SUMA(A2:A4;B4)"
tokenizer = Tokenizer(formula)
tokens = tokenizer.tokenize()
print(tokens)
parser = Parser(tokens)
#formula_tokens = ['1.05', '+', 'A1', '*', '(', '(', 'SUMA', '(', 'A2', ':', 'B5', ';', 'PROMEDIO', '(', 'B6', ':', 'D8', ')', ';', 'C1', ';', '27', ')', '/', '4', ')', '+', '(', 'D6', '-', 'D8', ')', ')']
gen_postfix = GeneratePostfix(tokens)
postfix_expression = gen_postfix.generate_postfix()
print(postfix_expression)

# # Example usage
# formula = "=1.05 + A1*((SUMA(A2:B5;PROMEDIO(B6:D8);C1;27)/4)+(D6-D8))"
# tokenizer = Tokenizer(formula)
# tokens = tokenizer.tokenize()
# print(tokens)
# parser = Parser(tokens)
# try:
#     parsed_tokens = parser.parse()
#     print("Syntax is correct:", parsed_tokens)
# except SyntaxError as e:
#     print("Syntax error:", e)
# generator = GeneratePostfix(tokens)
# postfix_expression = generator.shunting_yard()
# print("Postfix Expression:", postfix_expression)


['A4', '-', 'B2', '/', 'SUMA', '(', 'A2', ':', 'A4', ';', 'B4', ')']
['A4', 'B2', 'A2', 'A4', ':', 'B4', ';', 'SUMA', '/', '-']
